In [1]:
# -*- coding: utf-8 -*-
# @Time    : 2019/05
# @Author  : zhangle
# @function   : data mining
# @Software: Jupyter

import pandas as pd
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

data= pd.read_csv('C://Users//zhangle//Desktop//data_set//train_1.csv')
train = data.iloc[:543990,:]
test = data.iloc[-200000:,:]
print(train.shape)
print(test.shape)

D:\anocanda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(543990, 27)
(200000, 27)


In [2]:
#数据预处理
train = train[train.gender != '\\N']
test = test[test.gender != '\\N']
train['gender'] = train['gender'].apply(lambda x : int(x))
test['gender'] = test['gender'].apply(lambda x : int(x))

train = train[train.age != '\\N']
test = test[test.age != '\\N']
train['age'] = train['age'].apply(lambda x : int(x))
test['age'] = test['age'].apply(lambda x : int(x))

train = train[train['2_total_fee'] != '\\N']
test = test[test['2_total_fee'] != '\\N']
test.loc[test['2_total_fee'] == '\\N','2_total_fee'] = 0.0
train['2_total_fee'] = train['2_total_fee'].apply(lambda x : float(x))
test['2_total_fee'] = test['2_total_fee'].apply(lambda x : float(x))

train = train[train['3_total_fee'] != '\\N']
test = test[test['3_total_fee'] != '\\N']
test.loc[test['3_total_fee'] == '\\N','3_total_fee'] = 0.0
train['3_total_fee'] = train['3_total_fee'].apply(lambda x : float(x))
test['3_total_fee'] = test['3_total_fee'].apply(lambda x : float(x))
data = pd.concat([train, test]).reset_index(drop=True).fillna(0) #合并之后重新建立索引 将空缺的地方补为0
data.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,108.100000,0.000000,564.366667,1,31,0,0,0.0,99999825,Gx4sJzcQog01UhZL
1,1,0,10,265.20,261.20,208.50,174.50,0.000000,1,0,...,240.100000,355.166667,0.000000,1,30,0,0,0.0,90063345,kEXrhTiug93DIcLG
2,1,0,12,44.50,70.20,69.00,61.40,2598.397406,0,0,...,27.666667,0.000000,98.416667,2,25,0,0,0.0,90109916,AouXr0EOUtSRdiYK
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,89.900000,74.483333,121.833333,1,44,0,0,0.0,89950166,Yds7U30hnRZDiLtb
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,0.000000,35.100000,496.733333,1,31,2,1,0.0,99999825,OFDTSXrhN9Q2mbVw


In [3]:
test_Internet = test[test['current_service'].isin([90063345,90155946,90109916])]
train_Internet = train[train['current_service'].isin([90063345,90155946,90109916])]
train_ordinary = train[~train['current_service'].isin([90063345,90155946,90109916])]
test_ordinary = test[~test['current_service'].isin([90063345,90155946,90109916])]

In [4]:
def LGB():
        clf = lgb.LGBMClassifier(
                bjective='multiclass',
                boosting_type='gbdt',
                num_leaves=35,
                max_depth=8,
                learning_rate=0.05,
                seed=2018,
                colsample_bytree=0.8,
                subsample=0.9,
                n_estimators=10000)
        return clf

In [5]:
#准备 8套餐 训练集
Xtrain_ordinary = train_ordinary.drop(['user_id'],axis = 1)
Xtrain_feature_ordinary = [value for value in Xtrain_ordinary.columns.values if
                   value not in ['current_service']]
Xtrain_ordinary[Xtrain_feature_ordinary].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,4096.000000,1392.038508,108.100000,0.000000,564.366667,1,31,0,0,0.0
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,37.336425,988.561075,89.900000,74.483333,121.833333,1,44,0,0,0.0
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,3305.741127,0.000000,0.000000,35.100000,496.733333,1,31,2,1,0.0
5,4,0,46,36.00,36.00,36.00,37.20,218.003452,0,3,...,459.294048,218.003452,14.633333,0.000000,14.633333,1,42,0,0,0.0
7,4,0,109,116.00,116.00,116.00,116.00,635.946276,0,9,...,1024.000000,635.978400,250.883333,0.000000,310.733333,1,40,0,0,0.0


In [6]:
#全模型标签

label_ordinary = Xtrain_ordinary['current_service']
le = LabelEncoder()
label_ordinary = le.fit_transform(label_ordinary)
print(set(label_ordinary))

{0, 1, 2, 3, 4, 5, 6, 7}


In [7]:
#全模型测试集
Xtest_ordinary = test_ordinary.drop(['user_id'],axis = 1)
Xtest_feature_ordinary  = [value for value in Xtest_ordinary.columns.values if
                   value not in ['current_service']]
Xtest_ordinary[Xtest_feature_ordinary].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
543991,4,0,14,62.00,183.30,43.60,67.76,548.944325,0,0,...,0.000000,0.000000,0.000000,0.000000,51.466667,2,17,0,0,0.0
543996,4,0,32,196.00,256.00,286.00,232.00,1474.221327,0,1,...,0.000000,64.400747,0.000000,0.000000,15.466667,1,27,0,0,0.0
543997,4,0,87,86.00,79.45,76.00,76.00,1745.176379,0,0,...,70.994985,1140.720259,22.516667,0.000000,56.183333,2,32,0,0,0.0
543998,4,0,175,291.26,288.91,240.08,269.10,4381.679605,1,9,...,0.000000,2548.636515,418.100000,5.833333,696.316667,1,46,0,0,0.0
543999,4,0,134,76.00,75.94,76.00,75.95,228.412859,0,0,...,114.097425,228.412859,45.900000,0.000000,66.050000,1,33,0,0,0.0


In [8]:
#8模型训练
model = LGB()
model.fit(Xtrain_ordinary[Xtrain_feature_ordinary], label_ordinary, eval_set=[(Xtrain_ordinary[Xtrain_feature_ordinary], label_ordinary)], verbose=1)
pred1 = model.predict(Xtest_ordinary[Xtest_feature_ordinary])
pred1 = le.inverse_transform(pred1)
test_ordinary['predict'] = pred1

[1]	valid_0's multi_logloss: 1.75291
[2]	valid_0's multi_logloss: 1.66636
[3]	valid_0's multi_logloss: 1.59294
[4]	valid_0's multi_logloss: 1.52798
[5]	valid_0's multi_logloss: 1.46992
[6]	valid_0's multi_logloss: 1.41654
[7]	valid_0's multi_logloss: 1.36707
[8]	valid_0's multi_logloss: 1.32144
[9]	valid_0's multi_logloss: 1.28027
[10]	valid_0's multi_logloss: 1.24203
[11]	valid_0's multi_logloss: 1.20852
[12]	valid_0's multi_logloss: 1.17554
[13]	valid_0's multi_logloss: 1.14445
[14]	valid_0's multi_logloss: 1.11567
[15]	valid_0's multi_logloss: 1.08743
[16]	valid_0's multi_logloss: 1.06124
[17]	valid_0's multi_logloss: 1.03704
[18]	valid_0's multi_logloss: 1.01356
[19]	valid_0's multi_logloss: 0.991586
[20]	valid_0's multi_logloss: 0.970713
[21]	valid_0's multi_logloss: 0.950601
[22]	valid_0's multi_logloss: 0.931076
[23]	valid_0's multi_logloss: 0.913527
[24]	valid_0's multi_logloss: 0.896594
[25]	valid_0's multi_logloss: 0.879475
[26]	valid_0's multi_logloss: 0.863348
[27]	valid_0'

[210]	valid_0's multi_logloss: 0.376046
[211]	valid_0's multi_logloss: 0.375577
[212]	valid_0's multi_logloss: 0.375126
[213]	valid_0's multi_logloss: 0.374679
[214]	valid_0's multi_logloss: 0.374239
[215]	valid_0's multi_logloss: 0.373856
[216]	valid_0's multi_logloss: 0.373428
[217]	valid_0's multi_logloss: 0.372988
[218]	valid_0's multi_logloss: 0.372601
[219]	valid_0's multi_logloss: 0.372243
[220]	valid_0's multi_logloss: 0.371834
[221]	valid_0's multi_logloss: 0.371367
[222]	valid_0's multi_logloss: 0.370927
[223]	valid_0's multi_logloss: 0.37052
[224]	valid_0's multi_logloss: 0.370079
[225]	valid_0's multi_logloss: 0.369708
[226]	valid_0's multi_logloss: 0.369298
[227]	valid_0's multi_logloss: 0.368883
[228]	valid_0's multi_logloss: 0.36848
[229]	valid_0's multi_logloss: 0.368105
[230]	valid_0's multi_logloss: 0.367661
[231]	valid_0's multi_logloss: 0.367244
[232]	valid_0's multi_logloss: 0.366855
[233]	valid_0's multi_logloss: 0.366493
[234]	valid_0's multi_logloss: 0.366112
[2

[416]	valid_0's multi_logloss: 0.316337
[417]	valid_0's multi_logloss: 0.316142
[418]	valid_0's multi_logloss: 0.315964
[419]	valid_0's multi_logloss: 0.315758
[420]	valid_0's multi_logloss: 0.31558
[421]	valid_0's multi_logloss: 0.315397
[422]	valid_0's multi_logloss: 0.315205
[423]	valid_0's multi_logloss: 0.315032
[424]	valid_0's multi_logloss: 0.314844
[425]	valid_0's multi_logloss: 0.314634
[426]	valid_0's multi_logloss: 0.314451
[427]	valid_0's multi_logloss: 0.314248
[428]	valid_0's multi_logloss: 0.314049
[429]	valid_0's multi_logloss: 0.313874
[430]	valid_0's multi_logloss: 0.313705
[431]	valid_0's multi_logloss: 0.313506
[432]	valid_0's multi_logloss: 0.313328
[433]	valid_0's multi_logloss: 0.313131
[434]	valid_0's multi_logloss: 0.312944
[435]	valid_0's multi_logloss: 0.312753
[436]	valid_0's multi_logloss: 0.312577
[437]	valid_0's multi_logloss: 0.312383
[438]	valid_0's multi_logloss: 0.312206
[439]	valid_0's multi_logloss: 0.312026
[440]	valid_0's multi_logloss: 0.311833
[

[622]	valid_0's multi_logloss: 0.283727
[623]	valid_0's multi_logloss: 0.283589
[624]	valid_0's multi_logloss: 0.283463
[625]	valid_0's multi_logloss: 0.283293
[626]	valid_0's multi_logloss: 0.283166
[627]	valid_0's multi_logloss: 0.283046
[628]	valid_0's multi_logloss: 0.282892
[629]	valid_0's multi_logloss: 0.282761
[630]	valid_0's multi_logloss: 0.282618
[631]	valid_0's multi_logloss: 0.282507
[632]	valid_0's multi_logloss: 0.282397
[633]	valid_0's multi_logloss: 0.282259
[634]	valid_0's multi_logloss: 0.282128
[635]	valid_0's multi_logloss: 0.282005
[636]	valid_0's multi_logloss: 0.281868
[637]	valid_0's multi_logloss: 0.281724
[638]	valid_0's multi_logloss: 0.281603
[639]	valid_0's multi_logloss: 0.28149
[640]	valid_0's multi_logloss: 0.281361
[641]	valid_0's multi_logloss: 0.28121
[642]	valid_0's multi_logloss: 0.281097
[643]	valid_0's multi_logloss: 0.280971
[644]	valid_0's multi_logloss: 0.280811
[645]	valid_0's multi_logloss: 0.280677
[646]	valid_0's multi_logloss: 0.280508
[6

[828]	valid_0's multi_logloss: 0.259084
[829]	valid_0's multi_logloss: 0.258986
[830]	valid_0's multi_logloss: 0.258863
[831]	valid_0's multi_logloss: 0.258742
[832]	valid_0's multi_logloss: 0.258645
[833]	valid_0's multi_logloss: 0.258543
[834]	valid_0's multi_logloss: 0.258432
[835]	valid_0's multi_logloss: 0.258339
[836]	valid_0's multi_logloss: 0.258223
[837]	valid_0's multi_logloss: 0.25811
[838]	valid_0's multi_logloss: 0.258007
[839]	valid_0's multi_logloss: 0.257912
[840]	valid_0's multi_logloss: 0.257809
[841]	valid_0's multi_logloss: 0.257707
[842]	valid_0's multi_logloss: 0.257609
[843]	valid_0's multi_logloss: 0.257502
[844]	valid_0's multi_logloss: 0.257397
[845]	valid_0's multi_logloss: 0.25729
[846]	valid_0's multi_logloss: 0.257191
[847]	valid_0's multi_logloss: 0.2571
[848]	valid_0's multi_logloss: 0.256979
[849]	valid_0's multi_logloss: 0.256875
[850]	valid_0's multi_logloss: 0.256793
[851]	valid_0's multi_logloss: 0.25671
[852]	valid_0's multi_logloss: 0.256611
[853]

...

[9334]	valid_0's multi_logloss: 0.0207459
[9335]	valid_0's multi_logloss: 0.0207403
[9336]	valid_0's multi_logloss: 0.0207325
[9337]	valid_0's multi_logloss: 0.0207264
[9338]	valid_0's multi_logloss: 0.0207202
[9339]	valid_0's multi_logloss: 0.0207138
[9340]	valid_0's multi_logloss: 0.0207077
[9341]	valid_0's multi_logloss: 0.0207032
[9342]	valid_0's multi_logloss: 0.0206973
[9343]	valid_0's multi_logloss: 0.0206917
[9344]	valid_0's multi_logloss: 0.0206844
[9345]	valid_0's multi_logloss: 0.0206801
[9346]	valid_0's multi_logloss: 0.0206735
[9347]	valid_0's multi_logloss: 0.0206685
[9348]	valid_0's multi_logloss: 0.0206612
[9349]	valid_0's multi_logloss: 0.0206563
[9350]	valid_0's multi_logloss: 0.0206521
[9351]	valid_0's multi_logloss: 0.0206468
[9352]	valid_0's multi_logloss: 0.0206421
[9353]	valid_0's multi_logloss: 0.0206352
[9354]	valid_0's multi_logloss: 0.0206292
[9355]	valid_0's multi_logloss: 0.0206235
[9356]	valid_0's multi_logloss: 0.0206177
[9357]	valid_0's multi_logloss: 0.

[9530]	valid_0's multi_logloss: 0.0195516
[9531]	valid_0's multi_logloss: 0.0195471
[9532]	valid_0's multi_logloss: 0.0195434
[9533]	valid_0's multi_logloss: 0.0195375
[9534]	valid_0's multi_logloss: 0.0195315
[9535]	valid_0's multi_logloss: 0.0195254
[9536]	valid_0's multi_logloss: 0.0195172
[9537]	valid_0's multi_logloss: 0.019512
[9538]	valid_0's multi_logloss: 0.019508
[9539]	valid_0's multi_logloss: 0.0195028
[9540]	valid_0's multi_logloss: 0.0194959
[9541]	valid_0's multi_logloss: 0.01949
[9542]	valid_0's multi_logloss: 0.0194822
[9543]	valid_0's multi_logloss: 0.0194772
[9544]	valid_0's multi_logloss: 0.0194737
[9545]	valid_0's multi_logloss: 0.0194672
[9546]	valid_0's multi_logloss: 0.0194623
[9547]	valid_0's multi_logloss: 0.0194571
[9548]	valid_0's multi_logloss: 0.019451
[9549]	valid_0's multi_logloss: 0.0194452
[9550]	valid_0's multi_logloss: 0.0194397
[9551]	valid_0's multi_logloss: 0.0194344
[9552]	valid_0's multi_logloss: 0.0194292
[9553]	valid_0's multi_logloss: 0.01942

[9726]	valid_0's multi_logloss: 0.0184538
[9727]	valid_0's multi_logloss: 0.0184482
[9728]	valid_0's multi_logloss: 0.0184408
[9729]	valid_0's multi_logloss: 0.0184356
[9730]	valid_0's multi_logloss: 0.0184295
[9731]	valid_0's multi_logloss: 0.0184247
[9732]	valid_0's multi_logloss: 0.0184196
[9733]	valid_0's multi_logloss: 0.0184144
[9734]	valid_0's multi_logloss: 0.0184098
[9735]	valid_0's multi_logloss: 0.0184041
[9736]	valid_0's multi_logloss: 0.0183987
[9737]	valid_0's multi_logloss: 0.0183954
[9738]	valid_0's multi_logloss: 0.0183893
[9739]	valid_0's multi_logloss: 0.0183838
[9740]	valid_0's multi_logloss: 0.0183804
[9741]	valid_0's multi_logloss: 0.0183753
[9742]	valid_0's multi_logloss: 0.01837
[9743]	valid_0's multi_logloss: 0.0183654
[9744]	valid_0's multi_logloss: 0.0183617
[9745]	valid_0's multi_logloss: 0.0183586
[9746]	valid_0's multi_logloss: 0.0183543
[9747]	valid_0's multi_logloss: 0.0183491
[9748]	valid_0's multi_logloss: 0.0183447
[9749]	valid_0's multi_logloss: 0.01

[9922]	valid_0's multi_logloss: 0.0174536
[9923]	valid_0's multi_logloss: 0.017448
[9924]	valid_0's multi_logloss: 0.0174423
[9925]	valid_0's multi_logloss: 0.0174382
[9926]	valid_0's multi_logloss: 0.0174345
[9927]	valid_0's multi_logloss: 0.0174289
[9928]	valid_0's multi_logloss: 0.0174252
[9929]	valid_0's multi_logloss: 0.0174189
[9930]	valid_0's multi_logloss: 0.0174136
[9931]	valid_0's multi_logloss: 0.0174087
[9932]	valid_0's multi_logloss: 0.0174027
[9933]	valid_0's multi_logloss: 0.0173978
[9934]	valid_0's multi_logloss: 0.0173917
[9935]	valid_0's multi_logloss: 0.0173873
[9936]	valid_0's multi_logloss: 0.0173799
[9937]	valid_0's multi_logloss: 0.0173744
[9938]	valid_0's multi_logloss: 0.0173683
[9939]	valid_0's multi_logloss: 0.0173618
[9940]	valid_0's multi_logloss: 0.0173555
[9941]	valid_0's multi_logloss: 0.0173502
[9942]	valid_0's multi_logloss: 0.0173455
[9943]	valid_0's multi_logloss: 0.0173406
[9944]	valid_0's multi_logloss: 0.0173369
[9945]	valid_0's multi_logloss: 0.0

D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#8套餐预测显示
print(set(test_ordinary['predict']))
test_ordinary.head()

{99999825, 99999826, 99999827, 99999828, 89950167, 99999830, 89950166, 89950168}


,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543991,4,0,14,62.00,183.30,43.60,67.76,548.944325,0,0,...,0.000000,51.466667,2,17,0,0,0.0,89950168,6il0f5jstE3DIUAb,89950168
543996,4,0,32,196.00,256.00,286.00,232.00,1474.221327,0,1,...,0.000000,15.466667,1,27,0,0,0.0,99999826,6IlHVr8JimNXgGTk,99999826
543997,4,0,87,86.00,79.45,76.00,76.00,1745.176379,0,0,...,0.000000,56.183333,2,32,0,0,0.0,89950166,6iLJtBZqprc9y20X,89950166
543998,4,0,175,291.26,288.91,240.08,269.10,4381.679605,1,9,...,5.833333,696.316667,1,46,0,0,0.0,99999826,6IlmzTeEBJa1XyYD,99999827
543999,4,0,134,76.00,75.94,76.00,75.95,228.412859,0,0,...,0.000000,66.050000,1,33,0,0,0.0,99999830,6ilwz0s9nHaDU2fx,99999830


In [10]:
#准备 3套餐 训练集
Xtrain_Internet = train_Internet.drop(['user_id'],axis = 1)
Xtrain_feature_Internet = [value for value in Xtrain_Internet.columns.values if
                   value not in ['current_service']]
#全模型标签

label_Internet = Xtrain_Internet['current_service']
le = LabelEncoder()
label_Internet = le.fit_transform(label_Internet)
#全模型测试集
Xtest_Internet = test_Internet.drop(['user_id'],axis = 1)
Xtest_feature_Internet = [value for value in Xtest_Internet.columns.values if
                   value not in ['current_service']]
#全模型训练
model2 = LGB()
model2.fit(Xtrain_Internet[Xtrain_feature_Internet], label_Internet, eval_set=[(Xtrain_Internet[Xtrain_feature_Internet], label_Internet)], verbose=1)
pred2 = model2.predict(Xtest_Internet[Xtest_feature_Internet])
pred2 = le.inverse_transform(pred2)
test_Internet['predict'] = pred2


[1]	valid_0's multi_logloss: 0.515522
[2]	valid_0's multi_logloss: 0.465862
[3]	valid_0's multi_logloss: 0.431714
[4]	valid_0's multi_logloss: 0.399369
[5]	valid_0's multi_logloss: 0.373116
[6]	valid_0's multi_logloss: 0.349376
[7]	valid_0's multi_logloss: 0.32715
[8]	valid_0's multi_logloss: 0.308008
[9]	valid_0's multi_logloss: 0.289019
[10]	valid_0's multi_logloss: 0.272175
[11]	valid_0's multi_logloss: 0.25812
[12]	valid_0's multi_logloss: 0.244097
[13]	valid_0's multi_logloss: 0.23118
[14]	valid_0's multi_logloss: 0.219842
[15]	valid_0's multi_logloss: 0.208274
[16]	valid_0's multi_logloss: 0.197632
[17]	valid_0's multi_logloss: 0.187996
[18]	valid_0's multi_logloss: 0.178919
[19]	valid_0's multi_logloss: 0.17038
[20]	valid_0's multi_logloss: 0.162502
[21]	valid_0's multi_logloss: 0.155516
[22]	valid_0's multi_logloss: 0.148314
[23]	valid_0's multi_logloss: 0.141698
[24]	valid_0's multi_logloss: 0.13547
[25]	valid_0's multi_logloss: 0.129632
[26]	valid_0's multi_logloss: 0.124444


[206]	valid_0's multi_logloss: 0.00918807
[207]	valid_0's multi_logloss: 0.00914864
[208]	valid_0's multi_logloss: 0.00910705
[209]	valid_0's multi_logloss: 0.00907224
[210]	valid_0's multi_logloss: 0.00903229
[211]	valid_0's multi_logloss: 0.00899665
[212]	valid_0's multi_logloss: 0.0089615
[213]	valid_0's multi_logloss: 0.00892455
[214]	valid_0's multi_logloss: 0.0088881
[215]	valid_0's multi_logloss: 0.00885525
[216]	valid_0's multi_logloss: 0.0088202
[217]	valid_0's multi_logloss: 0.00878587
[218]	valid_0's multi_logloss: 0.00874868
[219]	valid_0's multi_logloss: 0.00871765
[220]	valid_0's multi_logloss: 0.00868274
[221]	valid_0's multi_logloss: 0.00865263
[222]	valid_0's multi_logloss: 0.0086198
[223]	valid_0's multi_logloss: 0.00858857
[224]	valid_0's multi_logloss: 0.00855475
[225]	valid_0's multi_logloss: 0.00852011
[226]	valid_0's multi_logloss: 0.0084889
[227]	valid_0's multi_logloss: 0.00845804
[228]	valid_0's multi_logloss: 0.00842465
[229]	valid_0's multi_logloss: 0.008393

[404]	valid_0's multi_logloss: 0.00509126
[405]	valid_0's multi_logloss: 0.00507724
[406]	valid_0's multi_logloss: 0.00506751
[407]	valid_0's multi_logloss: 0.00505648
[408]	valid_0's multi_logloss: 0.00504192
[409]	valid_0's multi_logloss: 0.00502782
[410]	valid_0's multi_logloss: 0.00501536
[411]	valid_0's multi_logloss: 0.00499942
[412]	valid_0's multi_logloss: 0.00498842
[413]	valid_0's multi_logloss: 0.00497905
[414]	valid_0's multi_logloss: 0.00496884
[415]	valid_0's multi_logloss: 0.00495563
[416]	valid_0's multi_logloss: 0.00494477
[417]	valid_0's multi_logloss: 0.00493445
[418]	valid_0's multi_logloss: 0.0049214
[419]	valid_0's multi_logloss: 0.00491071
[420]	valid_0's multi_logloss: 0.00489785
[421]	valid_0's multi_logloss: 0.00488381
[422]	valid_0's multi_logloss: 0.00487568
[423]	valid_0's multi_logloss: 0.00486047
[424]	valid_0's multi_logloss: 0.00484654
[425]	valid_0's multi_logloss: 0.00483645
[426]	valid_0's multi_logloss: 0.00482253
[427]	valid_0's multi_logloss: 0.00

[601]	valid_0's multi_logloss: 0.00329522
[602]	valid_0's multi_logloss: 0.00328632
[603]	valid_0's multi_logloss: 0.00328216
[604]	valid_0's multi_logloss: 0.00327484
[605]	valid_0's multi_logloss: 0.00326835
[606]	valid_0's multi_logloss: 0.00326149
[607]	valid_0's multi_logloss: 0.00325136
[608]	valid_0's multi_logloss: 0.0032447
[609]	valid_0's multi_logloss: 0.00323707
[610]	valid_0's multi_logloss: 0.00323044
[611]	valid_0's multi_logloss: 0.00322499
[612]	valid_0's multi_logloss: 0.00321607
[613]	valid_0's multi_logloss: 0.00320895
[614]	valid_0's multi_logloss: 0.00319985
[615]	valid_0's multi_logloss: 0.00319231
[616]	valid_0's multi_logloss: 0.00318413
[617]	valid_0's multi_logloss: 0.00317656
[618]	valid_0's multi_logloss: 0.00316805
[619]	valid_0's multi_logloss: 0.00316136
[620]	valid_0's multi_logloss: 0.0031525
[621]	valid_0's multi_logloss: 0.00314845
[622]	valid_0's multi_logloss: 0.00314226
[623]	valid_0's multi_logloss: 0.00313854
[624]	valid_0's multi_logloss: 0.003

[798]	valid_0's multi_logloss: 0.00216732
[799]	valid_0's multi_logloss: 0.00216352
[800]	valid_0's multi_logloss: 0.00215955
[801]	valid_0's multi_logloss: 0.00215496
[802]	valid_0's multi_logloss: 0.0021506
[803]	valid_0's multi_logloss: 0.00214563
[804]	valid_0's multi_logloss: 0.00214163
[805]	valid_0's multi_logloss: 0.0021353
[806]	valid_0's multi_logloss: 0.00212952
[807]	valid_0's multi_logloss: 0.00212508
[808]	valid_0's multi_logloss: 0.00212115
[809]	valid_0's multi_logloss: 0.0021166
[810]	valid_0's multi_logloss: 0.00211238
[811]	valid_0's multi_logloss: 0.00210698
[812]	valid_0's multi_logloss: 0.00210286
[813]	valid_0's multi_logloss: 0.00209746
[814]	valid_0's multi_logloss: 0.00209365
[815]	valid_0's multi_logloss: 0.00208945
[816]	valid_0's multi_logloss: 0.00208386
[817]	valid_0's multi_logloss: 0.00207874
[818]	valid_0's multi_logloss: 0.00207353
[819]	valid_0's multi_logloss: 0.002067
[820]	valid_0's multi_logloss: 0.00206302
[821]	valid_0's multi_logloss: 0.002059

...

[9435]	valid_0's multi_logloss: 4.58861e-08
[9436]	valid_0's multi_logloss: 4.58782e-08
[9437]	valid_0's multi_logloss: 4.58676e-08
[9438]	valid_0's multi_logloss: 4.58541e-08
[9439]	valid_0's multi_logloss: 4.58424e-08
[9440]	valid_0's multi_logloss: 4.58353e-08
[9441]	valid_0's multi_logloss: 4.58266e-08
[9442]	valid_0's multi_logloss: 4.58194e-08
[9443]	valid_0's multi_logloss: 4.58108e-08
[9444]	valid_0's multi_logloss: 4.58055e-08
[9445]	valid_0's multi_logloss: 4.57977e-08
[9446]	valid_0's multi_logloss: 4.57882e-08
[9447]	valid_0's multi_logloss: 4.57799e-08
[9448]	valid_0's multi_logloss: 4.57757e-08
[9449]	valid_0's multi_logloss: 4.57683e-08
[9450]	valid_0's multi_logloss: 4.57578e-08
[9451]	valid_0's multi_logloss: 4.5745e-08
[9452]	valid_0's multi_logloss: 4.57377e-08
[9453]	valid_0's multi_logloss: 4.57271e-08
[9454]	valid_0's multi_logloss: 4.57201e-08
[9455]	valid_0's multi_logloss: 4.57147e-08
[9456]	valid_0's multi_logloss: 4.57068e-08
[9457]	valid_0's multi_logloss: 4

[9623]	valid_0's multi_logloss: 4.43851e-08
[9624]	valid_0's multi_logloss: 4.43766e-08
[9625]	valid_0's multi_logloss: 4.43694e-08
[9626]	valid_0's multi_logloss: 4.43635e-08
[9627]	valid_0's multi_logloss: 4.43582e-08
[9628]	valid_0's multi_logloss: 4.43458e-08
[9629]	valid_0's multi_logloss: 4.43388e-08
[9630]	valid_0's multi_logloss: 4.43345e-08
[9631]	valid_0's multi_logloss: 4.43289e-08
[9632]	valid_0's multi_logloss: 4.43234e-08
[9633]	valid_0's multi_logloss: 4.43172e-08
[9634]	valid_0's multi_logloss: 4.4313e-08
[9635]	valid_0's multi_logloss: 4.43027e-08
[9636]	valid_0's multi_logloss: 4.42961e-08
[9637]	valid_0's multi_logloss: 4.42888e-08
[9638]	valid_0's multi_logloss: 4.42838e-08
[9639]	valid_0's multi_logloss: 4.42766e-08
[9640]	valid_0's multi_logloss: 4.42672e-08
[9641]	valid_0's multi_logloss: 4.42589e-08
[9642]	valid_0's multi_logloss: 4.42486e-08
[9643]	valid_0's multi_logloss: 4.42412e-08
[9644]	valid_0's multi_logloss: 4.42335e-08
[9645]	valid_0's multi_logloss: 4

[9811]	valid_0's multi_logloss: 4.29787e-08
[9812]	valid_0's multi_logloss: 4.29718e-08
[9813]	valid_0's multi_logloss: 4.29616e-08
[9814]	valid_0's multi_logloss: 4.29539e-08
[9815]	valid_0's multi_logloss: 4.29492e-08
[9816]	valid_0's multi_logloss: 4.29423e-08
[9817]	valid_0's multi_logloss: 4.29342e-08
[9818]	valid_0's multi_logloss: 4.29283e-08
[9819]	valid_0's multi_logloss: 4.29203e-08
[9820]	valid_0's multi_logloss: 4.29157e-08
[9821]	valid_0's multi_logloss: 4.29079e-08
[9822]	valid_0's multi_logloss: 4.29026e-08
[9823]	valid_0's multi_logloss: 4.28953e-08
[9824]	valid_0's multi_logloss: 4.28884e-08
[9825]	valid_0's multi_logloss: 4.28807e-08
[9826]	valid_0's multi_logloss: 4.28694e-08
[9827]	valid_0's multi_logloss: 4.28622e-08
[9828]	valid_0's multi_logloss: 4.28514e-08
[9829]	valid_0's multi_logloss: 4.28453e-08
[9830]	valid_0's multi_logloss: 4.28365e-08
[9831]	valid_0's multi_logloss: 4.28314e-08
[9832]	valid_0's multi_logloss: 4.28223e-08
[9833]	valid_0's multi_logloss: 

[9999]	valid_0's multi_logloss: 4.16716e-08
[10000]	valid_0's multi_logloss: 4.16632e-08


D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
test_new = pd.concat([test_Internet,test_ordinary],axis = 0)
test_new

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543990,1,0,15,49.10,50.90,44.50,57.60,0.000000,1,0,...,0.000000,6.666667,1,30,0,0,0.0,90063345,6Iky7Crmi9U2oeYc,90063345
543992,1,0,6,36.00,36.00,36.00,43.70,2135.174574,0,0,...,0.000000,87.233333,1,22,0,0,0.0,90109916,6iL7nFvTSUDXpsfw,90109916
543993,1,0,7,78.50,58.00,45.50,51.10,0.000000,1,0,...,78.800000,4.766667,2,23,0,0,0.0,90063345,6iL9jYKA0gBNPDWd,90063345
543994,1,0,4,60.40,64.90,33.29,0.00,0.000000,1,0,...,86.800000,0.000000,2,17,0,0,0.0,90063345,6ilDfetAok14JhZ0,90063345
543995,1,0,5,41.30,46.33,10.44,23.55,0.000000,1,0,...,8.650000,0.000000,1,38,0,0,0.0,90063345,6ilEz3sJkRTtDma4,90063345
544002,1,0,18,64.10,59.90,65.30,66.40,100.000000,1,0,...,72.350000,0.000000,1,23,0,0,0.0,90063345,6ImgF0Xk15of8ETj,90063345
544005,1,0,4,53.40,50.60,45.00,9.01,0.000000,1,0,...,6.950000,1.600000,1,28,0,0,0.0,90063345,6In0ZKNo8lCiVbRs,90063345
544007,1,0,15,32.30,36.30,33.30,29.70,0.000000,1,0,...,0.666667,0.000000,1,20,0,0,0.0,90063345,6INQARmDUiYhbas1,90063345
544010,1,0,13,53.10,54.70,48.70,56.30,0.000000,1,0,...,0.000000,0.000000,1,27,0,0,0.0,90063345,6IOdjRPo8NcZ5rfz,90063345
544012,1,0,17,62.20,88.70,66.10,75.30,1024.000000,1,0,...,97.183333,0.000000,1,21,0,0,0.0,90063345,6ioq4Rd9wpDxrHkt,90063345


In [12]:
#F1与准确率
y_true = test_new['current_service'].tolist()
y_pred = test_new['predict'].tolist()
print("8+3 LightGBM F1 Score:",f1_score(y_true, y_pred, average='macro'))
acc = 0
for i in range(len(y_pred)):
    if(y_pred[i] ==y_true[i]):
        acc+=1
print("8+3 LightGBM Accuracy:",acc/len(y_pred))

8+3 LightGBM F1 Score: 0.9159273559437419
8+3 LightGBM Accuracy: 0.9466694666946669


In [ ]:
#观察特征

In [98]:
service = set(test['current_service'])
print(service)

{90155946, 90063345, 99999826, 99999827, 99999828, 99999825, 89950166, 99999830, 89950168, 89950167, 90109916}


In [108]:
test_service = test['current_service'].groupby(test['predict'])
print(test_service.count())

predict
89950166    38193
89950167    18568
89950168     8307
90063345    77071
90109916    10249
90155946     5854
99999825     5882
99999826     8673
99999827     8557
99999828    13408
99999830     5236
Name: current_service, dtype: int64


In [125]:
df = pd.DataFrame()
df = test_service.count().reset_index()
df

,predict,current_service
0,89950166,38193
1,89950167,18568
2,89950168,8307
3,90063345,77071
4,90109916,10249
5,90155946,5854
6,99999825,5882
7,99999826,8673
8,99999827,8557
9,99999828,13408


In [136]:
ab = df['current_service']
ab.tolist()

[38193, 18568, 8307, 77071, 10249, 5854, 5882, 8673, 8557, 13408, 5236]

In [131]:
cur_ser = test['current_service'].tolist()
pre = test['predict'].tolist()

In [135]:
cntall = []
for i in ab:
    cnt = 0
    for j in range(len(cur_ser)):
        if cur_ser[j] == i and pre[j] == i:
            cnt += 1
    cntall.append(cnt)
cntall        

[33040, 15186, 7885, 76711, 10166, 5530, 5125, 5939, 5563, 9924, 4039]

In [137]:
accuracy = []
for i in range(len(ab)):
    accuracy.append(cntall[i]/ab[i])
accuracy

[0.8650799884795644,
 0.8178586816027574,
 0.9491994703262309,
 0.9953289823669085,
 0.9919016489413601,
 0.9446532285616672,
 0.8713022781366881,
 0.6847688227833506,
 0.6501110202173659,
 0.7401551312649165,
 0.7713903743315508]